In [30]:
import numpy as np


data_size = 1000
np.random.seed(42)
data = np.random.randint(-100, 100, size=data_size)

def modified_relu(data, embed_size=5):

    result = np.empty(data_size)

    alice = np.random.randint(-100, 100, size=data_size)
    bob = data - alice

    alice_mask = alice >= 0
    bob_mask = bob >= 0

    false_indices = np.where(alice_mask != bob_mask)[0]
    true_indices = np.where(alice_mask == bob_mask)[0]
    same_result = alice_mask[true_indices] + bob_mask[true_indices]

    keys = np.random.randint(0, embed_size, size=len(false_indices))

    # print(keys)
    embedded_alice = np.random.randint(-100, 100, size=(embed_size, len(false_indices)))
    embedded_alice[keys, range(len(false_indices))] = alice[false_indices]

    embedded_bob = embedded_alice + bob[false_indices]

    mask = embedded_bob > 0
    dif_result = mask[keys, range(len(false_indices))]
    result[true_indices] = same_result
    result[false_indices] = dif_result

    print(f"Accuracy:{((data > 0) == result).sum() * 100 / len(data) : .2f}%")
    
    return result


modified_relu(data)

Accuracy: 100.00%


array([1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1.,
       1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1.,
       1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0.,
       0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0.,
       0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1.,
       1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1.,
       1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1.,
       0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1.,
       0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0.,
       0., 1., 1., 1., 1.

In [29]:
import numpy as np

def verify_results(mask,data):
    ground_mask = (data > 0).astype(int)
    #print(ground_mask)
    #print(mask)
    return np.array_equal(mask, ground_mask)


def recover_data(original_data,original_positions,shuffled_mask):
    original_mask = np.zeros_like(original_data)
    for i in range(original_data.size):
        original_index = int(original_positions[original_positions[:, i] != -1, i])
        original_mask[original_index] = shuffled_mask[original_positions[:, i] != -1, i]

    return original_mask


def My_Relu(data_size, embed_size=4):
    # sender
    Data_send = np.random.randint(-128, 127, size=(data_size,), dtype=np.int8)
    extended_data_send = np.vstack([Data_send, np.random.randint(-128, 127, (4, Data_send.size), dtype=np.int8)])

    print(extended_data_send)

    original_positions = np.vstack([np.arange(Data_send.size), np.full((4, Data_send.size), -1)])

    # print(original_positions)


    # Shuffle the data within each column, preserving the mapping
    for col in range(extended_data_send.shape[1]):
        combined = np.column_stack((extended_data_send[:, col], original_positions[:, col]))
        # print(combined)

        # Combines the original position at column (col) with the extended_data_send at (col)
        np.random.shuffle(combined)
        # print(combined)

        # Updates the extended data and original positions from the combined columns
        # Combined[:, 0] holds the values for the extended data send 
        # Combined[:, 1] holds the values for the original positions

        extended_data_send[:, col], original_positions[:, col] = combined[:, 0], combined[:, 1]

    # print(extended_data_send, original_positions)
    # Reciever
    Data_rec = np.random.randint(-128, 127, size=(data_size,), dtype=np.int8)
    print(f"Receiver Data: {Data_rec}\n")

    relu_extended_data_send = (extended_data_send + Data_rec).astype(np.int8)
    print(f"Sender + Receiver Data: \n{relu_extended_data_send}\n")

    all_mask = (relu_extended_data_send > 0).astype(int)
    print(all_mask)

    #print(relu_extended_data_send)
    #print(mask)
    # Recover the original 1D array using the positions
    mask = recover_data(Data_send,original_positions,all_mask)

    if verify_results(mask,Data_send + Data_rec):
        print("Results Matched!")


My_Relu(5)

[[  59  120   -6 -128  -27]
 [ -96   13  -56   99   39]
 [  -3  101  -19  -51  -71]
 [ -20  -82   84  -25  121]
 [ -44  -17   69  -20  -49]]
Receiver Data: [ 39   0   5 -79  74]

Sender + Receiver Data: 
[[  98  -17   89  -99   47]
 [  -5  120   -1   20   25]
 [  36  101   74 -104    3]
 [  19   13  -14   49  -61]
 [ -57  -82  -51  126  113]]

[[1 0 1 0 1]
 [0 1 0 1 1]
 [1 1 1 0 1]
 [1 1 0 1 0]
 [0 0 0 1 1]]
Results Matched!


/tmp/ipykernel_4158/3064686244.py:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  original_index = int(original_positions[original_positions[:, i] != -1, i])
/tmp/ipykernel_4158/3064686244.py:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  original_mask[original_index] = shuffled_mask[original_positions[:, i] != -1, i]


In [5]:
import crypten
import torch
from crypten.mpc import MPCTensor

# Initialize CrypTen
crypten.init()

# Define a custom MPCTensor class with a modified ReLU implementation
class CustomMPCTensor(MPCTensor):
    def relu(self):
        """Custom secure ReLU operation"""
        zero = crypten.cryptensor(torch.zeros_like(self))  # Secure zero tensor
        return crypten.where(self > zero, self, zero)  # Secure comparison-based ReLU

# Register the new MPCTensor type

# Function to test the custom ReLU
def test_custom_relu():
    # Create a tensor with both positive and negative values
    x = torch.tensor([-3.0, -1.0, 0.0, 2.0, 5.0])

    # Encrypt the tensor with our custom MPCTensor
    x_enc = CustomMPCTensor(x)

    # Apply secure ReLU using our custom implementation
    relu_result = x_enc.relu()

    # Decrypt and reveal the result
    relu_decrypted = relu_result.get_plain_text()
    print("Original Tensor:", x)
    print("ReLU Output:", relu_decrypted)

# Run the test
test_custom_relu()

/home/comet/anaconda3/envs/research_py/lib/python3.9/site-packages/crypten/__init__.py:64: RuntimeWarning: CrypTen is already initialized.
  warnings.warn("CrypTen is already initialized.", RuntimeWarning)
/tmp/ipykernel_3023/2966382227.py:12: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:311.)
  zero = crypten.cryptensor(torch.zeros_like(self))  # Secure zero tensor


NotImplementedError: CrypTen does not support torch function <built-in method zeros_like of type object at 0x793c05cca1c0>.